In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np

# Load and reshape the image
def load_image(image_path):
    # Load the image directly without normalization
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=1)  # Load as grayscale
    img = tf.image.resize(img, [512, 512])  # Ensure it's 512x512
    img = tf.expand_dims(img, axis=0)  # Add batch dimension
    return img


In [ ]:
def attention_block(x, filters):
    # f and g reduce the number of filters
    f = layers.Conv2D(filters // 8, (1, 1), strides=(1, 1), padding='same')(x)
    g = layers.Conv2D(filters // 8, (1, 1), strides=(1, 1), padding='same')(x)
    h = layers.Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(x)

    # Calculate attention map
    attention = layers.Add()([f, g])  # Element-wise sum
    attention = layers.Conv2D(filters, (1, 1), padding='same')(attention)  # Match filter count
    attention = layers.Activation('softmax')(attention)

    # Apply attention to the feature map
    out = layers.Multiply()([attention, h])

    return out


def build_autoencoder(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Encoder with attention
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = attention_block(x, 64)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = attention_block(x, 128)

    # Bottleneck
    latent = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    latent = attention_block(latent, 256)

    # Decoder
    x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(latent)
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    outputs = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    return Model(inputs, outputs)

# Autoencoder input shape is 512x512x1 (grayscale)
autoencoder = build_autoencoder((512, 512, 1))


In [ ]:
# Define GAN generator
def build_gan_generator(input_shape):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)

    latent = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)

    x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(latent)
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)

    outputs = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    return Model(inputs, outputs)

# Generator input shape is the masked region (4x4)
gan_generator = build_gan_generator((4, 4, 1))


In [ ]:
# Define GAN discriminator
def build_discriminator(input_shape):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, (3, 3), strides=2, activation='relu', padding='same')(inputs)
    x = layers.Conv2D(128, (3, 3), strides=2, activation='relu', padding='same')(x)

    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)  # Output real/fake

    return Model(inputs, outputs)

# Discriminator input shape is the masked region (4x4)
discriminator = build_discriminator((4, 4, 1))


In [ ]:
# Loss functions (SSIM + MSE)
def ssim_loss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))

def combined_loss(y_true, y_pred, alpha=0.8):
    mse_loss = tf.reduce_mean(tf.keras.losses.MSE(y_true, y_pred))
    return alpha * mse_loss + (1 - alpha) * ssim_loss(y_true, y_pred)


In [ ]:
# Masking function remains unchanged
def mask_image(image, mask_size=10):
    mask = np.ones(image.shape)
    for i in range(0, image.shape[1], mask_size):
        for j in range(0, image.shape[2], mask_size):
            if np.random.rand() > 0.5:  # Randomly select blocks to mask
                mask[:, i:i+mask_size, j:j+mask_size, :] = 0
    return image * mask  # Return masked image


# Training loop
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

def display_images(original_image, masked_image, ae_output, gan_input, gan_output, epoch):
    """
    Display and save original image, input to autoencoder, input to GAN (masked), and their respective outputs.
    """
    fig, axs = plt.subplots(1, 5, figsize=(20, 5))

    # Convert tensors to numpy arrays for displaying
    original_image_np = original_image.numpy()
    masked_image_np = masked_image.numpy()
    ae_output_np = ae_output.numpy()
    gan_input_np = gan_input.numpy()
    gan_output_np = gan_output.numpy()

    # Original Image
    axs[0].imshow(original_image_np[0].squeeze(), cmap='gray')
    axs[0].set_title('Original Image')
    axs[0].axis('off')

    # Autoencoder Input (entire clean image)
    axs[1].imshow(masked_image_np[0].squeeze(), cmap='gray')
    axs[1].set_title('Autoencoder Input (Masked)')
    axs[1].axis('off')

    # Autoencoder Output
    axs[2].imshow(ae_output_np[0].squeeze(), cmap='gray')
    axs[2].set_title('Autoencoder Output')
    axs[2].axis('off')

    # GAN Input (Masked region)
    axs[3].imshow(gan_input_np[0].squeeze(), cmap='gray')
    axs[3].set_title('GAN Input (Masked Region)')
    axs[3].axis('off')

    # GAN Output (Reconstructed masked region)
    axs[4].imshow(gan_output_np[0].squeeze(), cmap='gray')
    axs[4].set_title('GAN Output')
    axs[4].axis('off')

    # Save the figure
    plt.savefig(f"outputs/training_epoch_{epoch}.png")
    plt.show()


def train(autoencoder, generator, discriminator, image, epochs=10000, save_interval=100):
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

    # Store the original image for comparison
    original_image = tf.convert_to_tensor(image)  # Assuming the image is loaded as a NumPy array

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            # Generate mask and apply to the image
            masked_image = mask_image(image)

            # Autoencoder forward pass with masked image
            ae_output = autoencoder(masked_image)

            # Autoencoder loss: comparing reconstructed image with the original (unmasked) image
            ae_loss = combined_loss(original_image, ae_output)  # SSIM + MSE with the original image

            # Select a random region for GAN (example: middle region 128:132 is masked)
            masked_region = masked_image[:, 128:132, 128:132, :]  # Input to GAN
            original_region = original_image[:, 128:132, 128:132, :]  # True region to compare against

            # GAN forward pass: reconstructing the masked region
            gen_output = generator(masked_region)

            # Discriminator: real and fake output
            real_output = discriminator(original_region)
            fake_output = discriminator(gen_output)

            # GAN loss: Discriminator loss + Generator loss
            gan_loss_real = tf.losses.binary_crossentropy(tf.ones_like(real_output), real_output)
            gan_loss_fake = tf.losses.binary_crossentropy(tf.zeros_like(fake_output), fake_output)
            gan_loss = gan_loss_real + gan_loss_fake

            # Total loss (autoencoder + GAN)
            total_loss = ae_loss + gan_loss

        # Backpropagation: Compute gradients and apply to all models
        gradients = tape.gradient(total_loss, autoencoder.trainable_variables + generator.trainable_variables + discriminator.trainable_variables)
        optimizer.apply_gradients(zip(gradients, autoencoder.trainable_variables + generator.trainable_variables + discriminator.trainable_variables))

        # Print images and losses every 50 epochs
        if epoch % 50 == 0:
            print(f"Epoch {epoch}: Autoencoder Loss: {ae_loss.numpy()}, GAN Loss: {gan_loss.numpy()}")
            display_images(original_image, masked_image, ae_output, masked_region, gen_output, epoch)

        # Save models every 100 epochs
        if epoch % save_interval == 0:
            autoencoder.save(f"models/autoencoder_{epoch}.h5")
            generator.save(f"models/generator_{epoch}.h5")
            discriminator.save(f"models/discriminator_{epoch}.h5")


In [ ]:
# Masking function remains unchanged
import matplotlib.pyplot as plt
def mask_image(image, mask_size=4):
    mask = np.ones(image.shape)
    for i in range(0, image.shape[1], mask_size):
        for j in range(0, image.shape[2], mask_size):
            if np.random.rand() > 0.5:  # Randomly select blocks to mask
                mask[:, i:i+mask_size, j:j+mask_size, :] = 0
    return image * mask  # Return masked image
def display_images(original_image, masked_image, gan_output, masked_region, ae_output, epoch):
    fig, axs = plt.subplots(1, 5, figsize=(20, 5))
    
    # Original Image
    axs[0].imshow(np.squeeze(original_image), cmap='gray')
    axs[0].set_title('Original Image')
    axs[0].axis('off')

    # Masked Input for GAN
    axs[1].imshow(np.squeeze(masked_image), cmap='gray')
    axs[1].set_title('Masked Input for GAN')
    axs[1].axis('off')

    # GAN Output (Full Image Reconstruction)
    axs[2].imshow(np.squeeze(gan_output), cmap='gray')
    axs[2].set_title('GAN Output')
    axs[2].axis('off')

    # Masked Region Input for Autoencoder
    axs[3].imshow(np.squeeze(masked_region), cmap='gray')
    axs[3].set_title('Autoencoder Input (Masked Region)')
    axs[3].axis('off')

    # Autoencoder Output (Reconstructed Region)
    axs[4].imshow(np.squeeze(ae_output), cmap='gray')
    axs[4].set_title('Autoencoder Output')
    axs[4].axis('off')

    plt.show()
# Updated Training loop with swapped roles
def train(autoencoder, generator, discriminator, image, epochs=1000, save_interval=100):
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

    # Store the original image for comparison
    original_image = tf.convert_to_tensor(image)  # Assuming the image is loaded as a NumPy array

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            # Generate mask and apply to the image
            masked_image = mask_image(image)

            ### GAN now reconstructs the full image from the masked image ###
            gan_output = generator(masked_image)

            # GAN loss: comparing reconstructed full image with the original unmasked image
            gan_loss = combined_loss(original_image, gan_output)  # SSIM + MSE with the original full image

            ### Autoencoder now reconstructs the masked regions ###
            masked_region = masked_image[:, 128:132, 128:132, :]  # Input to Autoencoder
            original_region = original_image[:, 128:132, 128:132, :]  # True region to compare against

            ae_output = autoencoder(masked_region)

            # Autoencoder loss: comparing the reconstructed region with the original unmasked region
            real_output = discriminator(original_region)
            fake_output = discriminator(ae_output)

            ae_loss_real = tf.losses.binary_crossentropy(tf.ones_like(real_output), real_output)
            ae_loss_fake = tf.losses.binary_crossentropy(tf.zeros_like(fake_output), fake_output)
            ae_loss = ae_loss_real + ae_loss_fake

            # Total loss (GAN + Autoencoder)
            total_loss = gan_loss + ae_loss

        # Backpropagation: Compute gradients and apply to all models
        gradients = tape.gradient(total_loss, generator.trainable_variables + autoencoder.trainable_variables + discriminator.trainable_variables)
        optimizer.apply_gradients(zip(gradients, generator.trainable_variables + autoencoder.trainable_variables + discriminator.trainable_variables))

        # Print images and losses every 50 epochs
        if epoch % 50 == 0:
            print(f"Epoch {epoch}: GAN Loss (Full Image): {gan_loss.numpy()}, Autoencoder Loss (Region): {ae_loss.numpy()}")
            display_images(original_image, masked_image, gan_output, masked_region, ae_output, epoch)

        # Save models every 100 epochs
        if epoch % save_interval == 0:
            generator.save(f"models/generator_{epoch}.h5")
            autoencoder.save(f"models/autoencoder_{epoch}.h5")
            discriminator.save(f"models/discriminator_{epoch}.h5")

image_path = '61_processed_image_edit.png'
image = load_image(image_path)  # Define a function to load your image, e.g., using tf.keras.preprocessing.image.load_img
image = image/255.0

# Initialize the models
autoencoder = build_autoencoder((4, 4, 1))  # Change according to your model design
gan_generator = build_gan_generator((512, 512, 1))  # Change according to your model design
discriminator = build_discriminator((4, 4, 1))  # Change according to your model design

# Train the models
train(autoencoder, gan_generator, discriminator, image, epochs=10000, save_interval=100)


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Function to load and normalize the test images (infected brain and segmentation mask)
def load_test_images(infected_image_path, segment_image_path):
    # Load infected brain image and normalize it
    infected_image = tf.keras.preprocessing.image.load_img(infected_image_path, color_mode='grayscale', target_size=(512, 512))
    infected_image = np.array(infected_image).reshape(1, 512, 512, 1)
    infected_image = infected_image / 255.0  # Normalize to [0, 1]

    # Load segment mask (just for display, no processing)
    segment_image = tf.keras.preprocessing.image.load_img(segment_image_path, color_mode='grayscale', target_size=(512, 512))
    segment_image = np.array(segment_image).reshape(1, 512, 512, 1)

    return infected_image, segment_image

# Function to display the test results
def display_test_results(original_image, segment_image, reconstructed_image, difference_image):
    fig, axs = plt.subplots(1, 4, figsize=(20, 5))

    # Display original infected brain image
    axs[0].imshow(original_image.squeeze(), cmap='gray')
    axs[0].set_title('Original Infected Brain Image')
    axs[0].axis('off')

    # Display real segment mask
    axs[1].imshow(segment_image.squeeze(), cmap='gray')
    axs[1].set_title('Real Segment Mask')
    axs[1].axis('off')

    # Display reconstructed clean brain image
    axs[2].imshow(reconstructed_image.squeeze(), cmap='gray')
    axs[2].set_title('Reconstructed Clean Brain Image')
    axs[2].axis('off')

    # Display difference image (original - reconstructed)
    axs[3].imshow(difference_image.squeeze(), cmap='gray')
    axs[3].set_title('Difference Image (Anomaly)')
    axs[3].axis('off')

    plt.show()

# Function for testing the complete model (Autoencoder + GAN)
def test_model(generator, infected_image_path, segment_image_path):
    # Load the test images
    infected_image, real_segment = load_test_images(infected_image_path, segment_image_path)

    # Step 1: Use the generator to reconstruct the clean brain image from the original infected image
    reconstructed_image = generator.predict(infected_image)

    # Step 2: Calculate the difference between the original infected image and the reconstructed clean image
    difference_image = np.abs(infected_image - reconstructed_image)

    # Step 3: Display the original, mask, reconstructed, and difference images
    display_test_results(infected_image, real_segment, reconstructed_image, difference_image)

# Paths to the test images (replace these paths with actual test image paths)
infected_image_path = r"C:\Users\priya\Documents\DL project\test 2\processed_image.png"
segment_image_path = r"C:\Users\priya\Documents\DL project\test 2\processed_mask.png"

# Load the trained models
generator = tf.keras.models.load_model('models/generator_9900.h5')  # Adjust path if needed

# Run the test
test_model(generator, infected_image_path, segment_image_path)


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Function to load and normalize the test images (infected brain)
def load_test_images(infected_image_path):
    infected_image = tf.keras.preprocessing.image.load_img(infected_image_path, color_mode='grayscale', target_size=(512, 512))
    infected_image = np.array(infected_image).reshape(1, 512, 512, 1)
    infected_image = infected_image / 255.0  # Normalize to [0, 1]
    return infected_image

# Function to predict the masked area using the autoencoder
def predict_masked_area(autoencoder, image, window_size=4):
    h, w = image.shape[1:3]
    masked_output = np.zeros_like(image)

    # Create a list to hold all windows for batch processing
    windows = []
    indices = []

    for i in range(0, h, window_size):
        for j in range(0, w, window_size):
            # Define window boundaries
            window = image[:, i:i + window_size, j:j + window_size, :]
            windows.append(window)
            indices.append((i, j))

    # Convert list of windows to numpy array for batch processing
    windows = np.concatenate(windows, axis=0)  # Shape: (number_of_windows, window_size, window_size, 1)
    print(f"Processing {windows.shape[0]} windows...")

    # Predict in batch
    predicted_windows = autoencoder.predict(windows)

    # Place predicted windows back into the masked output
    for idx, (i, j) in enumerate(indices):
        masked_output[:, i:i + window_size, j:j + window_size, :] = predicted_windows[idx]

    return masked_output

# Function to display the test results
def display_test_results(original_image, autoencoder_output, gan_output, combined_image):
    fig, axs = plt.subplots(1, 4, figsize=(20, 5))

    axs[0].imshow(original_image.squeeze(), cmap='gray')
    axs[0].set_title('Original Infected Brain Image')
    axs[0].axis('off')

    axs[1].imshow(autoencoder_output.squeeze(), cmap='gray')
    axs[1].set_title('Autoencoder Output (Masked Area)')
    axs[1].axis('off')

    axs[2].imshow(gan_output.squeeze(), cmap='gray')
    axs[2].set_title('GAN Output (Clean Brain Image)')
    axs[2].axis('off')

    axs[3].imshow(combined_image.squeeze(), cmap='gray')
    axs[3].set_title('Combined Image (Autoencoder + GAN)')
    axs[3].axis('off')

    plt.show()

# Function for testing the complete model (Autoencoder + GAN)
def test_model(autoencoder, generator, infected_image_path):
    # Load the test image
    infected_image = load_test_images(infected_image_path)

    # Step 1: Use the autoencoder to predict the masked area
    autoencoder_output = predict_masked_area(autoencoder, infected_image)

    # Step 2: Use the GAN to predict the clean brain image from the original infected image
    gan_output = generator.predict(infected_image)

    # Step 3: Combine the autoencoder output and GAN output
    combined_image = autoencoder_output + gan_output  # Adjust as needed to combine images

    # Step 4: Display the results
    display_test_results(infected_image, autoencoder_output, gan_output, combined_image)

# Paths to the test images (replace these paths with actual test image paths)
infected_image_path = r"C:\Users\priya\Documents\DL project\test\processed_image_edit.png"

# Load the trained models
autoencoder = tf.keras.models.load_model('models/autoencoder_9900.h5')  # Adjust path if needed
generator = tf.keras.models.load_model('models/generator_9900.h5')  # Adjust path if needed

# Run the test
test_model(autoencoder, generator, infected_image_path)
